In [11]:
proficiencia = {}

In [24]:
import pandas as pd
import numpy as np
import random as rd
from math import ceil,floor,factorial
from scipy.optimize import minimize
Times = pd.read_excel('TIMES.xlsx')
Times.set_index('Número', inplace = True)
Times.fillna('NA', inplace = True)
Times['Porcentagem de Vitorias'] = pd.Series(np.random.randn(len(list(Times.Sigla))), index=Times.index)
Times_msi = ['RNG','KZ','FW','FNC','TL','EVS','SUP','GMB','KBM','R7','KLG','DW','ASC','PGM']
Times_mundial_inicio = ['EDG','INF','DW','G2','SUP','ASC','C9','DFM','KBM','GRX','GMB','KLG','AFS','FW','PVB','RNG','VIT','GEN','KT','TL','MAD','FNC','IG','100']
Times_mundial_grupos = ['AFS','FW','PVB','RNG','VIT','GEN','KT','TL','MAD','FNC','IG','100','EDG','G2','C9','GRX']
Times_mundial_quartas = ['RNG','KT','FNC','IG','EDG','G2','C9','AFS']
ACERTOS=0
JOGOS_TOTAIS = 0

In [25]:
def arrumadados(sheet,guia,vero = False):
    arquivo = pd.ExcelFile(sheet)
    liga = pd.read_excel(arquivo,guia-1)
    if not vero:
        liga = liga[3:]
    liga.fillna(0,inplace = True)
    liga.replace(0.7,0.8, inplace = True)
    return liga

vitorias= {}
jogos = {}
for sigla in list(Times.Sigla):
    vitorias[sigla]=0
    jogos[sigla]=0

In [26]:
#Declarando as tabelas de jogos de cada regiao
cblol_2019_1 = arrumadados('CBLOL_BRASIL_2019.xlsx',1,True)
a = list(cblol_2019_1.Winner)
a.extend(list(cblol_2019_1.Loser))
times_cblol_2019 = set(a)

lec_2019_1 = arrumadados('LEC_EUROPA_2019.xlsx',1,True)
a = list(lec_2019_1.Winner)
a.extend(list(lec_2019_1.Loser))
times_lec_2019 = set(a)

lcs_2019_1 = arrumadados('LCS_USA_2019.xlsx',1,True)
lcs_2019_1.replace('100',100, inplace = True)
a = list(lcs_2019_1.Winner)
a.extend(list(lcs_2019_1.Loser))
times_lcs_2019 = set(a)

lck_2019_1 = arrumadados('LCK_COREIA_2019.xlsx',1,True)
a = list(lck_2019_1.Winner)
a.extend(list(lck_2019_1.Loser))
times_lck_2019 = set(a)

lpl_2019_1 = arrumadados('LPL_CHINA_2019.xlsx',1,True)
a = list(lpl_2019_1.Winner)
a.extend(list(lpl_2019_1.Loser))
times_lpl_2019 = set(a)

In [31]:
def liga_usada(liga,times,sigla):
    global x0,b,bnds
    x0 = list(np.random.uniform(0.5,2.5,len(times)))
    b = (0.001,10.0)
    bnds = tuple([b]*len(times))
    return (liga,times,sigla)

def vero(x):
    y = 0
    proficiencia[sigla] = {}
    for time in times:
        proficiencia[sigla][time] = x[y]
        y+=1
    multiplicatorio = 1
    #Defino a constante de esquecimento k, obtida experimentalmente
    k = 0.075
    for game in range (0,liga.shape[0]):
        w = liga.iloc[game].Week
        pv = proficiencia[sigla][liga.iloc[game].Winner]
        pl = proficiencia[sigla][liga.iloc[game].Loser]
        cte = 1 - (max(list(liga.Week)) - w)*k
        multiplicatorio *= (pv/(pv+pl))**cte
    
    return -np.log(multiplicatorio)

def probabilidade(A,B,n):
    siglaA = list(Times[Times.Sigla==A]['Região'])[0]
    siglaB = list(Times[Times.Sigla==B]['Região'])[0]
    A_list = []
    B_list = []
    i,k = 0,floor(n/2)
    x = proficiencia[siglaA][A]/(proficiencia[siglaA][A]+proficiencia[siglaB][B])
    print("         {} vs {}".format(A,B))
    while (ceil(n/2)+i) <= n:
        if i == 0:
            A_list.append(x**(ceil(n/2))*(1-x)**(i))
            B_list.append(x**(i)*(1-x)**(ceil(n/2)))
        else:
            p = factorial(k)/(factorial(i)*factorial(k-i)) 
            A_list.append(p*x**(ceil(n/2))*(1-x)**(i))
            B_list.append(p*x**(i)*(1-x)**(ceil(n/2)))
        i+=1
        k+=1
    if n==1:
        print('{} tem {:.2f} % de chance de vitória'.format(A,100*sum(A_list)))
        print('Enquanto {} tem {:.2f} %.'.format(B,100*sum(B_list)))
    else:    
        print('{} tem {:.2f} % de chance de vitória.Sendo:'.format(A,100*sum(A_list)))
        for b in range(0,ceil(n/2)):
            print('{:.2f} % a de ser {}-{}'.format(100*A_list[b],ceil(n/2),b))
        print('Enquanto {} tem {:.2f} %. Sendo:'.format(B,100*sum(B_list)))
        for b in range(0,ceil(n/2)):
            print('{:.2f} % a de ser {}-{}'.format(100*B_list[b],ceil(n/2),b))
    
def restricao1(x):
    return x[0]-1        

def restricao2(x):
    return sum(x)-5

con1 = {'type':'eq','fun':restricao1}
con2 = {'type':'eq','fun':restricao2}
cons = [con2]

In [28]:
#Todas as tabelas atualizadas ate 21/03
liga,times,sigla = liga_usada(lpl_2019_1,times_lpl_2019,'CN')
sol = minimize(vero,x0,method = 'SLSQP',constraints = cons,bounds = bnds)
proficiencia[sigla]

{'BLG': 0.3305739020544722,
 'EDG': 0.3647818762191129,
 'FPX': 0.7273629315027068,
 'IG': 0.6887912712157451,
 'JDG': 0.2978659060016903,
 'LGD': 0.15721984407045697,
 'OMG': 0.11104046913883135,
 'RNG': 0.40687721906131724,
 'RW': 0.08407117158228328,
 'SDG': 0.3677142996142714,
 'SN': 0.15866659121335502,
 'SS': 0.16164273588113529,
 'TOP': 0.5339008773114218,
 'V5': 0.1447826668690415,
 'VG': 0.10479266172750276,
 'WE': 0.3599155914378178}

### Previsoes da Liga Coreana
#### Taxa de Acerto Atual: 18/24   75,00%

In [751]:
# SEM A CONSTANTE DE ESQUECIMENTO 7++

In [ ]:
# COM A CONSTANTE DE ESQUECIMENTO 5++

In [964]:
probabilidade('SKT','JAG',3) #OK

         SKT vs JAG
SKT tem 99.64 % de chance de vitória.Sendo:
93.12 % a de ser 2-0
6.52 % a de ser 2-1
Enquanto JAG tem 0.36 %. Sendo:
0.12 % a de ser 2-0
0.24 % a de ser 2-1


In [965]:
probabilidade('GEN','DWG',3) #OK

         GEN vs DWG
GEN tem 25.44 % de chance de vitória.Sendo:
10.87 % a de ser 2-0
14.57 % a de ser 2-1
Enquanto DWG tem 74.56 %. Sendo:
44.94 % a de ser 2-0
29.63 % a de ser 2-1


In [17]:
probabilidade('HLE','KT',3) #OK

         HLE vs KT
HLE tem 66.61 % de chance de vitória.Sendo:
37.53 % a de ser 2-0
29.08 % a de ser 2-1
Enquanto KT tem 33.39 %. Sendo:
15.00 % a de ser 2-0
18.38 % a de ser 2-1


In [18]:
probabilidade('KZ','GRF',3) #N

         KZ vs GRF
KZ tem 30.91 % de chance de vitória.Sendo:
13.68 % a de ser 2-0
17.24 % a de ser 2-1
Enquanto GRF tem 69.09 %. Sendo:
39.71 % a de ser 2-0
29.37 % a de ser 2-1


### -----------------------------------------------------------------------------------------------------------------------------------------------

### Previsoes da Liga Chinesa
#### Taxa de Acerto Atual: 30/40   75,00%

In [ ]:
# SEM A CONSTANTE DE ESQUECIMENTO 5++

In [29]:
probabilidade('WE','RW',3)

         WE vs RW
WE tem 90.60 % de chance de vitória.Sendo:
65.71 % a de ser 2-0
24.89 % a de ser 2-1
Enquanto RW tem 9.40 %. Sendo:
3.59 % a de ser 2-0
5.81 % a de ser 2-1


In [30]:
probabilidade('LGD','SS',3)

         LGD vs SS
LGD tem 48.96 % de chance de vitória.Sendo:
24.31 % a de ser 2-0
24.65 % a de ser 2-1
Enquanto SS tem 51.04 %. Sendo:
25.70 % a de ser 2-0
25.34 % a de ser 2-1


In [ ]:
# COM A CONSTANTE DE ESQUECIMENTO 9++

In [ ]:
probabilidade('WE','RW',3)

In [ ]:
probabilidade('LGD','SS',3)

### -----------------------------------------------------------------------------------------------------------------------------------------------

### Previsoes da Liga Norte Americana
#### Taxa de Acerto Atual: 23/31    74,19%

In [ ]:
# SEM A CONSTANTE DE ESQUECIMENTO 2++

In [ ]:
# COM A CONSTANTE DE ESQUECIMENTO 9++

### -----------------------------------------------------------------------------------------------------------------------------------------------

### Previsoes da Liga Europeia
#### Taxa de Acerto Atual: 16/26 61,53%

In [943]:
probabilidade('FNC','VIT',5)

         FNC vs VIT
FNC tem 85.59 % de chance de vitória.Sendo:
36.52 % a de ser 3-0
31.25 % a de ser 3-1
17.83 % a de ser 3-2
Enquanto VIT tem 14.41 %. Sendo:
2.32 % a de ser 3-0
4.98 % a de ser 3-1
7.11 % a de ser 3-2


In [945]:
probabilidade('SPY','SK',5)

         SPY vs SK
SPY tem 66.54 % de chance de vitória.Sendo:
20.55 % a de ser 3-0
25.27 % a de ser 3-1
20.72 % a de ser 3-2
Enquanto SK tem 33.46 %. Sendo:
6.88 % a de ser 3-0
12.19 % a de ser 3-1
14.39 % a de ser 3-2


### -----------------------------------------------------------------------------------------------------------------------------------------------

### Previsoes da Liga Brasileira
#### Taxa de Acerto Atual: 29/36    80,55%

In [739]:
# SEM A CONSTANTE DE ESQUECIMENTO 3++

In [ ]:
# COM A CONSTANTE DE ESQUECIMENTO 5++

In [878]:
probabilidade('FLA','CNB',5)

         FLA vs CNB
FLA tem 99.92 % de chance de vitória.Sendo:
87.39 % a de ser 3-0
11.52 % a de ser 3-1
1.01 % a de ser 3-2
Enquanto CNB tem 0.08 %. Sendo:
0.01 % a de ser 3-0
0.02 % a de ser 3-1
0.05 % a de ser 3-2


In [884]:
probabilidade('RDP','ITZ',5)

         RDP vs ITZ
RDP tem 31.80 % de chance de vitória.Sendo:
6.42 % a de ser 3-0
11.54 % a de ser 3-1
13.84 % a de ser 3-2
Enquanto ITZ tem 68.20 %. Sendo:
21.56 % a de ser 3-0
25.90 % a de ser 3-1
20.74 % a de ser 3-2


# Calculo da proficiencia relativa entre as regioes

In [551]:
#Criando um dicionario com todas as regioes
regions = {}
for reg in set(Times.Região): 
    regions[reg] = list(Times.Sigla[Times.Região == reg])
    
#Criando um com todos os mundiais e importando os respectivos jogos
    mundial = {}
for ano in range (2011,2019):
    mundial[ano] = arrumadados("MUNDIAIS.xlsx",ano-2010,True)

#Criando um dataframe com as regiões e suas proficiências em cada ano
Proficiencias_Relativas = pd.DataFrame(data = None, index = list(set(Times.Região)),columns = [2011,2012,2013,2014,2015,2016,2017,2018,2019])


In [553]:
#Criando uma função que adiciona a proficiência relativa de cada região no dataframe
def adicione(ano):
    m = min(proficiencia.values())
    M = max(proficiencia.values())
    for reg in set(Times.Região):
        if reg in list(proficiencia.keys()):
            Proficiencias_Relativas.loc[[reg],[ano]] = (proficiencia[reg]-m)/(M-m)
            
#Modificando a funcao original pra que se adeque ao objetivo proposto
def vero_reg(x):
    y = 0
    for time in times:
        proficiencia_reg[time] = x[y]
        y+=1
    multiplicatorio = 1
    for ano in range(2011,2019):
        cship = mundial[ano]
        coef = 1 - (2018-ano)/8
        for game in range (0,int(cship.shape[0])):
            pv = proficiencia_reg[cship.iloc[game].Winner]
            pl = proficiencia_reg[cship.iloc[game].Loser]
            multiplicatorio *= (pv/(pv+pl))**(coef)
    
    return -np.log(multiplicatorio)

def probabilidadeR(A,B):
    x = proficiencia_reg[A]/(proficiencia_reg[A]+proficiencia_reg[B])
    print("         {} vs {}".format(A,B))
    print("{} tem {:.2f} % de chance de vitória".format(A,100*x))
    print("Enquanto {} tem {:.2f} %".format(B,100*(1-x)))

In [554]:
for camp in mundial.values():
    for reg in set(Times.Região):
        for linha in range(0,camp.shape[0]):
            if camp.iloc[linha].Winner in regions[reg]:
                camp.iloc[linha].Winner = reg
            if camp.iloc[linha].Loser in regions[reg]:
                camp.iloc[linha].Loser = reg

In [532]:
proficiencia_reg = {}
cons = con1
liga,times,sigla = liga_usada(lck_2019_1,list(set(Times.Região)),'MUND')
sol = minimize(vero_reg,x0,method = 'SLSQP',constraints = cons,bounds = bnds)
proficiencia_reg

{'BR': 0.11340527773331911,
 'CIS': 0.674702043456038,
 'CN': 1.3065021478148244,
 'EU': 1.0,
 'JPN': 0.06729952276967999,
 'KR': 2.252599874444157,
 'LAN': 0.3412384251901959,
 'LAS': 0.07090566327481033,
 'NA': 0.8036978293355066,
 'OCEANIA': 0.08932550771918506,
 'SEA': 0.015643127392125797,
 'TRK': 0.2634733283402824,
 'TWHKMA': 0.455997806299471,
 'VN': 0.3427505228918947}

In [538]:
sorted(proficiencia_reg.items(), key=lambda x: x[1])

[('SEA', 0.015643127392125797),
 ('JPN', 0.06729952276967999),
 ('LAS', 0.07090566327481033),
 ('OCEANIA', 0.08932550771918506),
 ('BR', 0.11340527773331911),
 ('TRK', 0.2634733283402824),
 ('LAN', 0.3412384251901959),
 ('VN', 0.3427505228918947),
 ('TWHKMA', 0.455997806299471),
 ('CIS', 0.674702043456038),
 ('NA', 0.8036978293355066),
 ('EU', 1.0),
 ('CN', 1.3065021478148244),
 ('KR', 2.252599874444157)]

In [542]:
probabilidadeR('LAS','BR')

         LAS vs BR
LAS tem 38.47 % de chance de vitória
Enquanto BR tem 61.53 %


# Colocando todos os jogos juntos (?)

In [768]:
def teste(a,n):
    x = a
    i = 0
    A_list = []
    B_list = []
    while (ceil(n/2)+i) <= n:
        k = ceil(n/2)+i
        if i == 0:
            A_list.append(x**(ceil(n/2))*(1-x)**(i))
            B_list.append(x**(i)*(1-x)**(ceil(n/2)))
        else:
            p = factorial(k)/(factorial(i)*factorial(k-i)) - 1
            A_list.append(p*x**(ceil(n/2))*(1-x)**(i))
            B_list.append(p*x**(i)*(1-x)**(ceil(n/2)))
        i+=1
    for b in range(0,ceil(n/2)):
        print('A probabilidade de A ganhar de {}-{} é {} '.format(ceil(n/2),b,A_list[b]))
    for b in range(0,ceil(n/2)):
        print('A probabilidade de B ganhar de {}-{} é {} '.format(ceil(n/2),b,B_list[b]))
        
        